# Notebook 5 — Classical time series models (SARIMA & Prophet)

This notebook focuses on classical time series models applied to the French hourly electricity load:

- SARIMA (from `statsmodels`)
- Prophet (from Meta)

In contrast with Notebook 4 (which uses feature-based machine learning models such as Ridge, Random forest, and XGBoost), this notebook works directly with the time series structure of the data.

---

## Objectives

1. Load and prepare a clean, regular hourly time series.
2. Fit a SARIMA model and evaluate its forecasting performance.
3. Fit a Prophet model and evaluate its forecasting performance.
4. Compare these classical models to the machine learning models from Notebook 4.


## 1. Load and prepare the base time series

We first load the cleaned hourly load series used throughout the project.

Steps:
- Load `datetime` and `load_mw`.
- Convert timestamps to UTC to avoid daylight savings time issues.
- Remove timezone information (naive `datetime64[ns]` index).
- Ensure a strictly hourly frequency.
- Interpolate occasional missing hours to obtain a fully regular series.


In [2]:
import pandas as pd
import numpy as np

# Load the base dataset (from previous notebooks)
df_raw = pd.read_csv("../data/energy_france.csv")

# Parse datetime with Europe/Paris timezone
df_raw["datetime"] = pd.to_datetime(df_raw["datetime"], utc=True).dt.tz_convert("Europe/Paris")

# Keep only what we need for classical TS models
df_ts = df_raw[["datetime", "load_mw"]].copy()

# Convert to UTC and drop timezone for regular time series modeling
df_ts["datetime_utc"] = df_ts["datetime"].dt.tz_convert("UTC").dt.tz_localize(None)

# Set index to UTC datetime
df_ts = df_ts.set_index("datetime_utc").sort_index()

df_ts = df_ts[["load_mw"]]  # keep only target

print(df_ts.info())
df_ts.head()


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 50357 entries, 2015-01-01 01:00:00 to 2020-09-30 22:00:00
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   load_mw  50357 non-null  float64
dtypes: float64(1)
memory usage: 786.8 KB
None


,load_mw
datetime_utc,
2015-01-01 01:00:00,69773.0
2015-01-01 02:00:00,66417.0
2015-01-01 03:00:00,64182.0
2015-01-01 04:00:00,63859.0
2015-01-01 05:00:00,63921.0


We now have a single time series:

- index: `datetime_utc` (naive `datetime64[ns]`, hourly)
- column: `load_mw` (float), the national electricity demand in MW.

Next, we must verify that the series is strictly hourly with no missing timestamps.


## 2. Check regular hourly frequency and fill missing timestamps

Classical time series models such as SARIMA and Prophet assume a regular time grid.

We:
- reindex the series on a strict hourly range,
- detect missing timestamps,
- fill them by interpolation.


In [3]:
# Create a complete hourly index from min to max date (UTC)
full_index = pd.date_range(
    start=df_ts.index.min(),
    end=df_ts.index.max(),
    freq="h"
)

print("Original length:", len(df_ts))
print("Full expected length:", len(full_index))

# Reindex to the full range
df_ts_full = df_ts.reindex(full_index)

missing_count = df_ts_full["load_mw"].isna().sum()
print("Missing timestamps after reindex:", missing_count)

# Interpolate missing values linearly
df_ts_full["load_mw"] = df_ts_full["load_mw"].interpolate(
    method="linear",
    limit_direction="both"
)

print("Missing after interpolation:", df_ts_full["load_mw"].isna().sum())

df_ts_full.head()


Original length: 50357
Full expected length: 50398
Missing timestamps after reindex: 41
Missing after interpolation: 0


,load_mw
2015-01-01 01:00:00,69773.0
2015-01-01 02:00:00,66417.0
2015-01-01 03:00:00,64182.0
2015-01-01 04:00:00,63859.0
2015-01-01 05:00:00,63921.0


The reindexed series now has:

- a perfectly regular hourly index,
- no missing values after interpolation.

This series is ready to be used with SARIMA and Prophet.
For the rest of this notebook, `df_ts_full["load_mw"]` is our target time series.


## 3. Train/validation split (time-based)

To evaluate classical time series models in a realistic scenario, 
we split the data chronologically:

- Training period: from the beginning of the series up to 2018-12-31
- Validation period: from 2019-01-01 onward

It is split chronologically rather than randomly because random splits would leak future information into the past and produce overly optimistic metrics.

This setup ensures that all models are evaluated on truly unseen future data,
reflecting a realistic forecasting scenario. The validation window spans 2019–2020,
which includes a wide range of seasonal patterns and demand variations, making it a
suitable benchmark for assessing forecasting performance.


In [15]:
from datetime import datetime

SPLIT_DATE = datetime(2019, 1, 1)

y = df_ts_full["load_mw"].copy()
time_index = df_ts_full.index

train_mask = time_index < SPLIT_DATE
valid_mask = time_index >= SPLIT_DATE

y_train = y.loc[train_mask]
y_valid = y.loc[valid_mask]
t_train = time_index[train_mask]
t_valid = time_index[valid_mask]

print("Train length:", len(y_train))
print("Valid length:", len(y_valid))
print("Train period:", t_train.min(), "->", t_train.max())
print("Valid period:", t_valid.min(), "->", t_valid.max())


Train length: 35063
Valid length: 15335
Train period: 2015-01-01 01:00:00 -> 2018-12-31 23:00:00
Valid period: 2019-01-01 00:00:00 -> 2020-09-30 22:00:00


## 4. SARIMA model

We now fit a SARIMA model on the hourly load series.

We use a seasonal ARIMA structure with daily seasonality:

- Non-seasonal order: (p, d, q) = (1, 1, 1)
- Seasonal order: (P, D, Q, s) = (1, 1, 1, 24)

This specification is a reasonable starting point for hourly load data with a strong daily cycle.


In [5]:
import warnings
warnings.filterwarnings("ignore")

import statsmodels.api as sm

# Define SARIMA structure
sarima_order = (1, 1, 1)
seasonal_order = (1, 1, 1, 24)

# Train SARIMA on the training portion only
sarima_model = sm.tsa.SARIMAX(
    y_train,
    order=sarima_order,
    seasonal_order=seasonal_order,
    enforce_stationarity=False,
    enforce_invertibility=False
)

sarima_results = sarima_model.fit(disp=False)

# Display only the summary header (model name + AIC/BIC)
sarima_results.summary().tables[0]


Dep. Variable:,load_mw,No. Observations:,35063
Model:,"SARIMAX(1, 1, 1)x(1, 1, 1, 24)",Log Likelihood,-275608.081
Date:,"Tue, 25 Nov 2025",AIC,551226.162
Time:,08:56:35,BIC,551268.479
Sample:,01-01-2015,HQIC,551239.641
,- 12-31-2018,,
Covariance Type:,opg,,


**Interpretation of the SARIMAX model summary**

The table above summarizes the fitted SARIMA(1,1,1)(1,1,1,24) model.

Key points:
- Number of observations (35,063) corresponds to the entire training period
  from January 2015 to December 2018.
- The model includes both non-seasonal and seasonal components:
  - (1,1,1) captures short-term autocorrelation and trend differencing.
  - (1,1,1,24) models the strong 24-hour seasonal cycle typical of hourly electricity load.
- The **log-likelihood**, **AIC**, **BIC**, and **HQIC** values quantify the fit of the model.
  These metrics are primarily useful when comparing different SARIMA configurations.
  Lower values indicate a better statistical fit.
- The covariance type “opg” refers to the method used to estimate parameter uncertainty.

At this stage, the model fit is complete.  
The next step is to generate out-of-sample forecasts for the validation period and
evaluate how well SARIMA predicts actual electricity load values.


### Forecasting the validation period
  
We now forecast the entire validation period starting on 2019-01-01.

SARIMA generates forecasts based on internal time indexing, so we specify:
- `start` = end of the training window
- `end`   = end of the validation window

The resulting forecast series will then be realigned with the timestamps of the
validation set.


In [6]:
# Numeric range for SARIMA forecasting
start = len(y_train)
end = len(y_train) + len(y_valid) - 1

# Forecast the entire validation period
sarima_forecast = sarima_results.predict(start=start, end=end)

# Align timestamps with the validation index
sarima_forecast.index = y_valid.index

# Preview the first few predictions
sarima_forecast.head()


2019-01-01 00:00:00    60150.231371
2019-01-01 01:00:00    58756.889224
2019-01-01 02:00:00    56142.649244
2019-01-01 03:00:00    55081.090428
2019-01-01 04:00:00    56392.138184
Freq: h, Name: predicted_mean, dtype: float64

**Interpretation of the first SARIMA predictions**

The values above correspond to the first hourly forecasts produced by the
SARIMA(1,1,1)(1,1,1,24) model for the start of the validation period
beginning on 2019-01-01.

Each value represents the model's estimate of the electricity load (in MW)
for a specific hour.

The next step is to evaluate these predictions quantitatively using RMSE
and MAPE, and later to visualize them over a short time window to assess
how closely SARIMA tracks the actual load curve.


### Evaluation of SARIMA forecasts

To assess the forecasting performance of the SARIMA model, we compare its predictions
to the actual load values on the validation set using two standard metrics:

- RMSE (Root Mean Squared Error)
  Measures the average magnitude of forecast errors.  
  Lower values indicate better predictive accuracy.

- MAPE (Mean Absolute Percentage Error)  
  Measures the average error relative to actual values, expressed as a percentage.  
  This metric is intuitive because it shows how far predictions deviate from real load values in percentage terms.

The goal of this step is to quantify how well SARIMA performs compared with
our earlier baselines and machine learning models.


In [ ]:
import numpy as np

# Define metrics
def rmse(y_true, y_pred):
    return np.sqrt(np.mean((np.asarray(y_true) - np.asarray(y_pred))**2))

def mape(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true)
    y_pred = np.asarray(y_pred)
    return np.mean(np.abs((y_true - y_pred) / (y_true + eps))) * 100

# Compute metrics
rmse_sarima = rmse(y_valid, sarima_forecast)
mape_sarima = mape(y_valid, sarima_forecast)

rmse_sarima, mape_sarima


(np.float64(89093.740095992), np.float64(170.09942814149855))

**SARIMA performance analysis**

The SARIMA(1,1,1)(1,1,1,24) model shows very poor forecasting performance on the
validation set:

- RMSE ≈ 89,094 MW  
- MAPE ≈ 170 %

These values indicate that the model is systematically far from the true load,
with errors several times larger than the actual signal. This confirms that
this SARIMA specification is not appropriate for our dataset.

Such a result is not unusual for high-frequency electricity load data.
Hourly load exhibits strong non-linearities, varying seasonal amplitudes,
and structural changes that are difficult for classical SARIMA models to capture.
Additionally, differencing the series (both regular and seasonal) can amplify
noise and lead to unstable forecasts if the model is not carefully tuned.

Overall, SARIMA provides a useful classical baseline, but in this case it is
clearly outperformed by feature-based machine learning models such as
Ridge regression, Random Forest, and XGBoost.


## 5. Prophet modeling

Prophet requires a specific data format with:
- a datetime column named ds
- a target column named y

In this step, we convert our cleaned time series (`df_ts_full`) into Prophet’s
required format. We keep the same train/validation split as in previous sections
to ensure consistency when comparing forecasting performance across models.


In [10]:
# Prepare data in Prophet format
df_prophet = df_ts_full.reset_index().rename(columns={
    "index": "ds",
    "load_mw": "y"
})

df_prophet["ds"] = pd.to_datetime(df_prophet["ds"])

# Reapply the same split date
from datetime import datetime

SPLIT_DATE = datetime(2019, 1, 1)

train_prophet = df_prophet[df_prophet["ds"] < SPLIT_DATE]
valid_prophet = df_prophet[df_prophet["ds"] >= SPLIT_DATE]

print("Train:", train_prophet.shape)
print("Valid:", valid_prophet.shape)
train_prophet.head()


Train: (35063, 2)
Valid: (15335, 2)


,ds,y
0,2015-01-01 01:00:00,69773.0
1,2015-01-01 02:00:00,66417.0
2,2015-01-01 03:00:00,64182.0
3,2015-01-01 04:00:00,63859.0
4,2015-01-01 05:00:00,63921.0


### Create and configure the Prophet model

Prophet decomposes the time series into trend, multiple seasonalities, and
holiday effects. Because our data is hourly, we explicitly enable:

- yearly seasonality
- weekly seasonality
- daily seasonality (important for hourly electricity load)

This configuration allows Prophet to model the main cycles present in the load
series without manual feature engineering.


In [11]:
from prophet import Prophet

# Create Prophet model with daily + weekly + yearly seasonality
model_prophet = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=True,
    daily_seasonality=True,
    changepoint_prior_scale=0.05  # default, controls trend flexibility
)

# Fit model on training data
model_prophet.fit(train_prophet)

model_prophet


09:36:54 - cmdstanpy - INFO - Chain [1] start processing
09:37:21 - cmdstanpy - INFO - Chain [1] done processing


### Forecasting the validation period with Prophet

Prophet requires a DataFrame containing the future timestamps to be predicted.
We therefore create a forecast frame using the `ds` column from the validation set,
and ask Prophet to generate predictions for each timestamp.

Prophet returns a full decomposition of the forecast (trend, seasonality, etc.).
For evaluation, we will extract the `yhat` column, which contains the point forecast.


In [12]:
# Build the prediction frame using the validation timestamps
future_valid = valid_prophet[["ds"]].copy()

# Generate predictions
forecast_prophet = model_prophet.predict(future_valid)

# Extract only the yhat predictions and align index
prophet_pred = forecast_prophet[["ds", "yhat"]].set_index("ds")["yhat"]

# Show first predictions
prophet_pred.head()


ds
2019-01-01 00:00:00    60122.505777
2019-01-01 01:00:00    57499.518586
2019-01-01 02:00:00    56061.076262
2019-01-01 03:00:00    56810.331670
2019-01-01 04:00:00    59579.474810
Name: yhat, dtype: float64

### Evaluation of Prophet forecasts

We now compare Prophet’s predictions to the actual load values in the
validation period using RMSE and MAPE.

These metrics will allow us to assess:
- how well Prophet captures the overall structure of the time series,
- and how its accuracy compares with previous models such as SARIMA,
  Ridge regression, Random Forest, and XGBoost.

Because Prophet internally models daily, weekly, and yearly seasonalities,
we expect it to outperform SARIMA and provide a competitive time-series-only baseline.


In [13]:
# Compute RMSE and MAPE for Prophet
rmse_prophet = rmse(valid_prophet["y"], prophet_pred)
mape_prophet = mape(valid_prophet["y"], prophet_pred)

rmse_prophet, mape_prophet


(np.float64(8437.601647853975), np.float64(15.163622229347837))

### Interpretation of Prophet performance

Prophet achieves the following results on the validation period:

- RMSE ≈ 8,438 MW
- MAPE ≈ 15.16 %

These values represent a significant improvement over the SARIMA model, which
performed very poorly on this dataset. Prophet’s ability to model multiple
seasonalities (daily, weekly, yearly) allows it to capture part of the structure
of the electricity load.

However, the error remains relatively high compared with the feature-based
machine learning models developed earlier (Ridge, Random Forest, XGBoost),
which achieved MAPE values between 1 % and 2 %. This gap illustrates a key
limitation of Prophet: although it is powerful for general time-series
structures, it does not incorporate engineered lag features or rolling-window
statistics, which are crucial for accurately modeling electricity demand.

Prophet therefore provides a solid classical baseline, but not a state-of-the-art
model for this specific forecasting task.
